# 00 FRED API and Caching

Fetch indicators from FRED and cache raw JSON.


## Table of Contents
- [Choose series](#choose-series)
- [Fetch metadata](#fetch-metadata)
- [Fetch + cache observations](#fetch-cache-observations)
- [Fallback to sample](#fallback-to-sample)
- [Checkpoint (Self-Check)](#checkpoint-self-check)
- [Solutions (Reference)](#solutions-reference)


## Why This Notebook Matters
Data notebooks build the datasets used everywhere else. If these steps are wrong, every model result is suspect.
You will practice:
- API ingestion and caching,
- frequency alignment,
- label construction.


## Prerequisites (Quick Self-Check)
- Completed Part 00 (foundations) or equivalent time-series basics.
- FRED API key set (`FRED_API_KEY`) for real data (sample data works offline).

## What You Will Produce
- (no file output; learning/analysis notebook)

## Success Criteria
- You can explain what you built and why each step exists.
- You can run your work end-to-end without undefined variables.

## Common Pitfalls
- Running cells top-to-bottom without reading the instructions.
- Leaving `...` placeholders in code cells.
- Merging mixed-frequency series without explicit resampling/aggregation.
- Forgetting to shift targets for forecasting tasks.

## Quick Fixes (When You Get Stuck)
- If you see `ModuleNotFoundError`, re-run the bootstrap cell and restart the kernel; make sure `PROJECT_ROOT` is the repo root.
- If a `data/processed/*` file is missing, either run the matching build script (see guide) or use the notebook’s `data/sample/*` fallback.
- If results look “too good,” suspect leakage; re-check shifts, rolling windows, and time splits.
- If a model errors, check dtypes (`astype(float)`) and missingness (`dropna()` on required columns).

## Matching Guide
- `docs/guides/01_data/00_fred_api_and_caching.md`



## How To Use This Notebook
- Work section-by-section; don’t skip the markdown.
- Most code cells are incomplete on purpose: replace TODOs and `...`, then run.
- After each section, write 2–4 sentences answering the interpretation prompts (what changed, why it matters).
- Prefer `data/processed/*` if you have built the real datasets; otherwise use the bundled `data/sample/*` fallbacks.
- Use the **Checkpoint (Self-Check)** section to catch mistakes early.
- Use **Solutions (Reference)** only to unblock yourself; then re-implement without looking.
- Use the matching guide (`docs/guides/01_data/00_fred_api_and_caching.md`) for the math, assumptions, and deeper context.



<a id="environment-bootstrap"></a>
## Environment Bootstrap
Run this cell first. It makes the repo importable and defines common directories.



In [ ]:
from __future__ import annotations

from pathlib import Path
import sys


def find_repo_root(start: Path) -> Path:
    p = start
    for _ in range(8):
        if (p / 'src').exists() and (p / 'docs').exists():
            return p
        p = p.parent
    raise RuntimeError('Could not find repo root. Start Jupyter from the repo root.')


PROJECT_ROOT = find_repo_root(Path.cwd())
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

DATA_DIR = PROJECT_ROOT / 'data'
RAW_DIR = DATA_DIR / 'raw'
PROCESSED_DIR = DATA_DIR / 'processed'
SAMPLE_DIR = DATA_DIR / 'sample'

PROJECT_ROOT



## Goal
Fetch a basket of economic indicators from FRED, cache the raw API responses, and build a tidy time series panel.

### Why this matters
You want to separate:
- **data acquisition** (API calls) from
- **analysis/modeling** (notebooks and scripts).

Caching makes your work reproducible and faster.



## Primer: Paths, files, and environment variables (how this repo stays reproducible)

You will see a few patterns repeatedly in notebooks and scripts.

### Environment variables (API keys)

Environment variables are key/value settings provided by your shell to Python.
This repo uses them for API keys:
- `FRED_API_KEY`
- `CENSUS_API_KEY` (optional)

```python
import os

fred_key = os.getenv("FRED_API_KEY")
print("FRED key set?", fred_key is not None)
```

If you set a key in a terminal, restart the Jupyter kernel so Python sees it.

### Paths (why `pathlib.Path` is the default)

Use `Path` to build OS-safe file paths:

```python
from pathlib import Path

p = Path("data") / "sample" / "macro_quarterly_sample.csv"
print(p, "exists?", p.exists())
```

### Repo bootstrap variables (defined in every notebook)

The notebook bootstrap cell defines:
- `PROJECT_ROOT` (repo root)
- `DATA_DIR`, `RAW_DIR`, `PROCESSED_DIR`, `SAMPLE_DIR`

Prefer these over hard-coded relative paths.

### Sample vs processed data (offline-first)

Most notebooks follow this pattern:
1) try `data/processed/*` (real pipeline output)
2) fall back to `data/sample/*` (small offline dataset)

This keeps notebooks runnable without network access.

### Common “file not found” fixes

- Print the path and check `.exists()`
- Print current working directory:
  - `import os; print(os.getcwd())`
- Start Jupyter from the repo root (so bootstrap can find `src/` and `docs/`)


<a id="choose-series"></a>
## Choose series

### Goal
Pick a starter basket of macro indicators.

### Notes
- Different indicators have different frequencies (monthly, daily, quarterly).
- We'll deal with alignment in later notebooks.



### Your Turn (1): Define a basket of series IDs


In [ ]:
# TODO: Define a list of FRED series IDs (strings)
# Suggested starters:
# - UNRATE   (Unemployment rate, monthly)
# - FEDFUNDS (Fed funds rate, monthly)
# - CPIAUCSL (CPI, monthly)
# - INDPRO   (Industrial Production, monthly)
# - RSAFS    (Retail Sales, monthly)
# - T10Y2Y   (10Y-2Y yield spread, daily)
series_ids = [
    ...,
]

# TODO: Basic validation
assert isinstance(series_ids, list)
assert all(isinstance(x, str) and x for x in series_ids)
assert len(series_ids) == len(set(series_ids)), 'duplicate series IDs'
series_ids



<a id="fetch-metadata"></a>
## Fetch metadata

### Goal
Metadata answers: what is this series, what are the units, and what is its native frequency?

You will use metadata later to decide:
- which features are meaningful,
- how to align frequencies,
- how to interpret coefficient units.



### Your Turn (1): Fetch metadata for one series


In [ ]:
from src import fred_api

# TODO: Pick one series_id and fetch metadata
sid = series_ids[0]
meta = fred_api.fetch_series_meta(sid)

# TODO: Print the most important fields (title, units, frequency)
...



### Your Turn (2): Fetch metadata for all series and build a table


In [ ]:
import pandas as pd
from src import fred_api

# TODO: Loop over series_ids and build a DataFrame of metadata.
# Hint: meta is a dict; you can select keys you care about.
rows = []
for sid in series_ids:
    meta = fred_api.fetch_series_meta(sid)
    rows.append({
        'id': sid,
        'title': meta.get('title'),
        'units': meta.get('units'),
        'frequency': meta.get('frequency'),
        'seasonal_adjustment': meta.get('seasonal_adjustment'),
    })

meta_df = pd.DataFrame(rows)
meta_df



<a id="fetch-cache-observations"></a>
## Fetch + cache observations

### Goal
Download observations for each series and cache the raw JSON under `data/raw/fred/`.

### Why cache raw JSON?
- It's the exact raw record of what the API returned.
- You can debug parsing issues later without re-downloading.



### Your Turn (1): Fetch and cache JSON payloads


In [ ]:
from src import data as data_utils
from src import fred_api

# We'll store raw API responses here
raw_dir = RAW_DIR / 'fred'
raw_dir.mkdir(parents=True, exist_ok=True)

# TODO: For each series_id, cache JSON under data/raw/fred/<id>.json
# Hint: data_utils.load_or_fetch_json(path, fetch_fn)
...



### Your Turn (2): Convert cached payloads to a tidy DataFrame panel


In [ ]:
import pandas as pd
from src import data as data_utils
from src import fred_api

# TODO: For each series, load JSON and convert to a 1-column DataFrame
# Hint: fred_api.observations_to_frame(payload, sid)
frames = []
for sid in series_ids:
    payload = data_utils.load_json(raw_dir / f'{sid}.json')
    frames.append(fred_api.observations_to_frame(payload, sid))

panel = pd.concat(frames, axis=1).sort_index()
panel.head()



### Your Turn (3): Inspect missingness and basic ranges


In [ ]:
# TODO: Print missing values per column
# TODO: Print min/max dates
# TODO: Describe each column
print('date range:', panel.index.min(), '->', panel.index.max())
print(panel.isna().sum().sort_values(ascending=False))
panel.describe().T



### Checkpoint (panel sanity)


In [ ]:
# TODO: These checks should pass if you built a valid panel.
assert isinstance(panel.index, pd.DatetimeIndex)
assert panel.index.is_monotonic_increasing
assert panel.shape[0] > 200
assert panel.shape[1] == len(series_ids)
...



<a id="fallback-to-sample"></a>
## Fallback to sample

If you cannot fetch from the API (no key, no network), load the bundled sample panel.



In [ ]:
import os
import pandas as pd

# TODO: Implement a fallback:
# - if FRED_API_KEY is missing, load data/sample/panel_monthly_sample.csv
# - otherwise, keep using your freshly built panel
if not os.getenv('FRED_API_KEY'):
    panel = pd.read_csv(SAMPLE_DIR / 'panel_monthly_sample.csv', index_col=0, parse_dates=True)

panel.head()



<a id="checkpoint-self-check"></a>
## Checkpoint (Self-Check)
Run a few asserts and write 2-3 sentences summarizing what you verified.



In [ ]:
import pandas as pd

# Expected file: (see notebook front matter)
# TODO: After saving your processed dataset, load it and run checks.
# df = pd.read_csv(PROCESSED_DIR / 'your_file.csv', index_col=0, parse_dates=True)
# assert df.index.is_monotonic_increasing
# assert df.shape[0] > 20
# print(df.dtypes)
...



## Extensions (Optional)
- Try one additional variant beyond the main path (different features, different split, different model).
- Write down what improved, what got worse, and your hypothesis for why.



## Reflection
- What did you assume implicitly (about timing, availability, stationarity, or costs)?
- If you had to ship this model, what would you monitor?



<a id="solutions-reference"></a>
## Solutions (Reference)

Try the TODOs first. Use these only to unblock yourself or to compare approaches.

<details><summary>Solution: Choose series</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 00_fred_api_and_caching — Choose series
series_ids = ['UNRATE', 'FEDFUNDS', 'CPIAUCSL', 'INDPRO', 'RSAFS', 'T10Y2Y']
series_ids
```

</details>

<details><summary>Solution: Fetch metadata</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 00_fred_api_and_caching — Fetch metadata
from src import fred_api

meta = fred_api.fetch_series_meta('UNRATE')
meta
```

</details>

<details><summary>Solution: Fetch + cache observations</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 00_fred_api_and_caching — Fetch + cache observations
from src import data as data_utils
from src import fred_api

raw_dir = RAW_DIR / 'fred'
raw_dir.mkdir(parents=True, exist_ok=True)

frames = []
for sid in series_ids:
    payload = data_utils.load_or_fetch_json(
        raw_dir / f'{sid}.json',
        lambda sid=sid: fred_api.fetch_series_observations(sid, start_date='1980-01-01', end_date=None),
    )
    frames.append(fred_api.observations_to_frame(payload, sid))

panel = pd.concat(frames, axis=1).sort_index()
panel.head()
```

</details>

<details><summary>Solution: Fallback to sample</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 00_fred_api_and_caching — Fallback to sample
import pandas as pd

panel = pd.read_csv(SAMPLE_DIR / 'panel_monthly_sample.csv', index_col=0, parse_dates=True)
panel.head()
```

</details>

